In [1]:
# load and autoreload
from IPython import get_ipython
# noinspection PyBroadException
try:
    _magic = get_ipython().magic
    _magic('load_ext autoreload')
    _magic('autoreload 2')
except:
    pass
    

import os 
import glob
import sys
import pprint
import datetime as dt
import pandas as pd 
import numpy as np
import matplotlib as mpl
import matplotlib.colors
import matplotlib.pyplot as plt 
import xarray as xr 
import seaborn as sns
import cartopy as crt

In [2]:
import requests
import urllib.parse

In [3]:
df = pd.read_csv('/Users/aliaga/Downloads/south.america.cities.mortality.txt')

In [4]:
cols = ['adm2_name','adm1_name','country_name']
d1 = df.drop_duplicates(cols )

In [5]:
res = {}

In [6]:
# bo = d1.index>263334

for i,r in d1[:].iterrows():
    
    f = f"{r['adm2_name']}, {r['adm1_name']}, {r['country_name']}"

    address = f
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
    response = requests.get(url).json()    
    elev,lat,lon = None,None,None
    if len(response)>0:

        lat = response[0]["lat"]
        lon = response[0]["lon"]


        url = f'https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}'

        response = requests.get(url).json()

        elev = response['results'][0]['elevation']
    res[i] = [elev,lat,lon]
    print(i)

0
1457
2792
4250
5691
6579
7359
8814
9689
11055
12402
13793
15066
16525
17953
19398
20792
22184
23290
24520
25980
27110
28434
29874
31288
32516
33976
35143
35323
36759
38097
39081
40101
41174
42603
44051
45394
46651
47996
49453
50719
52178
53635
55094
56547
58000
59189
60279
61738
62882
64045
64517
65007
66376
67466
68521
69358
70188
71198
72148
73083
74067
75393
76074
77039
78497
79119
80413
81552
82618
83506
84603
86062
87287
88706
89978
91299
92575
93939
95398
96844
98126
99582
100817
102223
103664
104905
106336
107543
108872
110330
111575
113031
114396
115254
115265
116649
118006
119427
120787
121895
123248
124670
126049
127504
128962
130224
131538
132570
132739
133899
134991
135729
137183
137789
138545
139188
140381
141512
142752
143011
143964
145077
146519
147059
147652
148487
149944
151403
152679
153959
155005
156201
157453
158484
159622
160927
162309
163612
165060
166085
167465
168812
170180
171537
172711
173824
174803
176212
177589
178776
180019
181386
182587
184040
185371
186

In [7]:
ser = pd.DataFrame(res,index= ['Elevation','Lat','Lon']).T
# ser.name = 'Elevation'

In [8]:
ser.isna().sum()

Elevation    4
Lat          4
Lon          4
dtype: int64

In [9]:
d2 = pd.merge(d1,ser,left_index=True,right_index=True)

In [10]:
d3 = pd.merge(d2[[*cols,'Elevation']],df,left_on=cols,right_on=cols)

In [11]:
d3.to_csv('./data/city_data_elev_ll.csv')